[py040 Питон - сырое чтение дисков](#py040-Питон---сырое-чтение-дисков)  
[py041 Питон сетевой обмен по UDP протоколу](#py041-Питон-сетевой-обмен-по-UDP-протоколу)   
[py042 Передача файлов по сети](#py042-Передача-файлов-по-сети)   
[py043 Питон программа управления шаговым двигателем](#py043-Питон-программа-управления-шаговым-двигателем)   
[py044 Питон многопоточные клиент и сервер UDP](#py044-Питон-многопоточные-клиент-и-сервер-UDP)   
[py045 Питон система ведущий-ведомые в локальной сети](#py045-Питон-система-ведущий-ведомые-в-локальной-сети)   
[py046 Питон парсинг HTML страницы](#py046-Питон-парсинг-HTML-страницы)   
[py047 Питон работа по Modbus RTU](#py047-Питон-работа-по-Modbus-RTU)   
[py048 Консольное приложение для работы по протоколу Modbus RTU](#py048-Консольное-приложение-для-работы-по-протоколу-Modbus-RTU)   
[py049 Питон управляем Экселем](#py049-Питон-управляем-Экселем)   


#### py040 Питон - сырое чтение дисков

In [2]:
%%writefile read.py
#coding: utf8
import os.path

file_name = ""
#-- определить список логических дисков
disk_lett = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
drives = ['%s:' % dsk for dsk in disk_lett if os.path.exists('%s:' % dsk)]
#print drives

#-- определить список физических дисков
hdd_num = 1
while True:
    hdd = r"\\.\PhysicalDrive" + str(hdd_num)    
    try:
        fh = open(hdd, 'rb')
        fh.close()
        hdd_num +=1
    except:
        break
hdd_count = hdd_num

#-- сформировать представление подстроки из байтов
def get_row(st_in, offset, row_len):
    st_loc = st_in[offset:offset + row_len]
    st_bt = ""
    st_ch = ''
    for num in range(len(st_loc)):
        ch = st_loc[num]
        bt = hex(ord(ch))
#         bt = hex(ch)
        bt = str(bt)[2:]
        if len(bt)<2: bt = '0'+bt
        st_bt = st_bt + bt + ' '
        if ord(ch)<32 or ord(ch)>126: ch = '.'
        st_ch = st_ch + ch + ' '
    return st_bt + st_ch

#-- прочитать и вывести сектор
def fn_read(ev):
    w = ev.widget
    #global file_name
    #-- прочитать нужный сектор
    sect_num = int(ed_sect.get())
    fh = open(file_name, 'rb')
    fh.seek(sect_num*512)
    sect2 = fh.read(512)
    sect=[]
    for i in sect2:
        sect.append(chr(i))
#     print(sect[0],print(type(sect[0])),sect)
    fh.close()
    
    #-- вывести его в текстовое поле
    tx_sect.delete('1.0', END)
    for row_num in range(32):
        offset = row_num * 16
        st = get_row(sect, offset, 16) +'\n'
        tx_sect.insert(END, st )

#-- нарисовать форму
from tkinter import *
root = Tk()

cl_contr = '#990000'

#== основные панели =========================================
#side-расположение, fill-заполнение, expand-расширение

#-- панель с органами управления 
pn_left = Frame(root, width = 200, bg = cl_contr)
pn_left.pack(side = 'left', fill = 'y')

#----> компоненты в панели с органами управления
##-- список с физическими дисками
def fn_lxhddselect(ev):
    global file_name
    w = ev.widget
    index = int(w.curselection()[0])
    file_name =  r"\\.\PhysicalDrive" + str(index)
    ed_disk.delete(0, END)
    ed_disk.insert(0, "HDD "+str(index))

lb_hdd = Label(pn_left, text = 'Физический диск:', bg = cl_contr, fg = 'white' )
lb_hdd.grid(row=0, column=0)
lx_hdd = Listbox(pn_left,  height = hdd_count )
lx_hdd.grid(row=1, column=0)
lx_hdd.bind('<<ListboxSelect>>', fn_lxhddselect)
for num in range(hdd_count):
    lx_hdd.insert(END, "HDD "+str(num))


Label(pn_left, text = ' ', bg = cl_contr ).grid(row=2, column=0)

##-- список с логическими дисками
def fn_lxdriveselect(ev):
    global file_name
    w = ev.widget
    index = int(w.curselection()[0])
    file_name =  "\\\\.\\" + drives[index]
    ed_disk.delete(0, END)
    ed_disk.insert(0, drives[index])

lb_drive = Label(pn_left, text = 'Логический диск:', bg = cl_contr, fg = 'white' )
lb_drive.grid(row=3, column=0)
lx_drive = Listbox(pn_left,  height = 5)
lx_drive.grid(row=4, column=0, padx= 10, pady= 10)
lx_drive.bind('<<ListboxSelect>>', fn_lxdriveselect)
for disk in drives:
    lx_drive.insert(END, disk)

Label(pn_left, text = ' ', bg = cl_contr ).grid(row=5, column=0)

##-- поле для выбранного диска
lb_disk = Label(pn_left, text = 'диск:', bg = cl_contr, fg = 'white' )
lb_disk.grid(row=6, column=0, padx= 10, pady= 10)
ed_disk = Entry(pn_left, width = 20)
ed_disk.grid(row=7, column=0)
#Label(pn_left, text = ' ', bg = cl_contr ).grid(row=8, column=0)

##-- поле для номера сектора
lb_sect = Label(pn_left, text = 'номер сектора:', bg = cl_contr, fg = 'white' )
lb_sect.grid(row=8, column=0, padx= 10, pady= 10)
ed_sect = Entry(pn_left, width = 20)
ed_sect.grid(row=9, column=0)
Label(pn_left, text = ' ', bg = cl_contr ).grid(row=11, column=0)

##-- кнопка для чтения сектора
bt_read = Button(pn_left, text = 'Читать')
bt_read.bind("<Button-1>", fn_read)
bt_read.grid(row = 10, column = 0, padx= 10, pady= 10)

#-- правая панель для текста сектора
pn_right = Frame(root, width = 800)
pn_right.pack(side = 'right', fill = 'both', expand = 1)

##-- поле для вывода текста в правой панели
tx_sect = Text(pn_right, height = 35, width = 84)
tx_sect.grid(row = 0, column = 0)

sb_hor = Scrollbar(pn_right, command = tx_sect.xview, orient = HORIZONTAL)  
sb_hor.grid(row = 1, column = 0, sticky = W+E+N+S)

sb_ver = Scrollbar(pn_right, command = tx_sect.yview, orient = VERTICAL)  
sb_ver.grid(row = 0, column = 1, sticky = W+E+N+S)

tx_sect.config(yscrollcommand = sb_ver.set, xscrollcommand = sb_hor.set, wrap = NONE)

#-- запустить окно программы
root.mainloop()



Writing read.py


#### py041 Питон сетевой обмен по UDP протоколу

In [14]:
%%writefile server.py
#server.py
from socket import *

HOST=''
PORT = 3050
BUFSIZE= 1024
SOCKADDR= (HOST, PORT)
uServSock = socket(AF_INET, SOCK_DGRAM)
uServSock.bind(SOCKADDR)
while True:
    print ('waiting:')
    data,addr = uServSock.recvfrom(BUFSIZE)
    loc_data = data.decode('cp1251')
    print ('received from" ', addr, ' data: ', loc_data)
    answer = 'answer to ' + addr[0] + ', echo: ' + loc_data
    uServSock.sendto(answer.encode(), addr)
uservSock.close()
    

Writing server.py


In [15]:
%%writefile client.py
#client.py
from socket import *
HOST = '127.0.0.1'
PORT=3050
BUFSIZE = 1024
SOCKADDR = (HOST,PORT)
uCliSock = socket(AF_INET,SOCK_DGRAM)
while True:
    data = input('>')
    if not data: break
    uCliSock.sendto(data.encode(), SOCKADDR)
    data, addr = uCliSock.recvfrom(BUFSIZE)
    if not data: break
    print (data)
uCliSock.close()
        

Writing client.py


[stackoverflow](https://ru.stackoverflow.com/questions/466658/%D0%9E%D1%88%D0%B8%D0%B1%D0%BA%D0%B0-%D0%BF%D1%80%D0%B8-%D0%BF%D0%BE%D0%B4%D0%BA%D0%BB%D1%8E%D1%87%D0%B5%D0%BD%D0%B8%D0%B8-%D1%81%D0%BE%D0%BA%D0%B5%D1%82%D0%B0-%D0%BA-%D1%81%D0%B5%D1%80%D0%B2%D0%B5%D1%80%D1%83-a-bytes-like-object-is-required-not-s)  
@Flaffen: нужно заметить, что код в вопросе работает без ошибок на Питоне 2, где str -- это последовательность байт. На Питоне 3, str -- это Юникодная строка (используется для текста). .encode(), .decode() используют sys.getdefaultencoding(), которая ascii на Питоне 2 и utf-8 на Питоне 3 по умолчанию. – jfs 16 ноя '15 в 0:50

#### py042 Передача файлов по сети
[pyftpdlib](https://pypi.org/project/pyftpdlib/)  
pip install pyftpdlib   


In [8]:
from pyftpdlib.servers import FTPServer
authorizer = FTPServer.DummyAuthorizer()
authorizer.add_user("ab", "12345", "/ftp_files", perm="alradfmw")
handler= FTPServer.FTPHandler
handler.authorizer = authorizer
address =("127.0.0.1", 21)
ftpd = ftpserver.FTPServer(address, handler)
ftpd.serve_forever()


AttributeError: type object 'FTPServer' has no attribute 'DummyAuthorizer'

In [1]:
%%writefile ftp_server.py
from pyftpdlib.authorizers import DummyAuthorizer
from pyftpdlib.handlers import FTPHandler
from pyftpdlib.servers import FTPServer

authorizer = DummyAuthorizer()
authorizer.add_user("user", "12345", "c:/ftp_files", perm="elradfmwMT")
# authorizer.add_anonymous("/home/nobody")

handler = FTPHandler
handler.authorizer = authorizer

server = FTPServer(("127.0.0.1", 21), handler)
server.serve_forever()


Writing ftp_server.py


In [ ]:
# import pyftpdlib
# print(help(pyftpdlib))

D:\mailCloud\prjother\089hosts\jnotebook\pyoop\prkt>ftp
ftp> ?
Допускается сокращение команд при вводе.  Набор команд:

!               delete          literal         prompt          send
?               debug           ls              put             status
append          dir             mdelete         pwd             trace
ascii           disconnect      mdir            quit            type
bell            get             mget            quote           user
binary          glob            mkdir           recv            verbose
bye             hash            mls             remotehelp
cd              help            mput            rename
close           lcd             open            rmdir
ftp> open 127.0.0.1
Уже установлена связь с "127.0.0.1", необходимо выполнить отключение.
ftp> user
Имя пользователя user
331 Username ok, send password.
Пароль:
230 Login successful.
ftp> ls
200 Active data connection established.
125 Data connection already open. Transfer starting.
A.txt
226 Transfer complete.
ftp: 7 байт получено за 0,01 (сек) со скоростью 1,40 (КБ/сек).


get,bput
##### ftp client

In [4]:
%%writefile ftp_client.py
# coding: cp1251 
import ftplib 
HOST = '127.0.0.1'

def put_file(param):
    print ('put_file: ', param) 
    file_name = param 
    ftp = ftplib.FTP(HOST) 
    ftp.login("user", "12345") 
    try:
        ftp.storbinary("STOR " + file_name, open(file_name, "rb"), 1024) 
        ftp.quit()
    except:
        print("ftp error: :("); return -1
    return 1

def get_file(param):
    print('get_file: '), param 
    file_name = param 
    ftp = ftplib.FTP(HOST) 
    ftp.login("user", "12345") 
    try:
        ftp.retrbinary("RETR " + file_name, open(file_name, "wb").write) 
        ftp.quit()
    except:
        print("ftp error: :(");return -1
    return 1

while True:
    data = input('>') 
    ls_data = data.split() 
    cmd = ls_data[0]
    param = ls_data[1]
    
    if(cmd == 'get'):
        res = get_file(param)
    if(cmd == 'put'):
        res = put_file(param)
        
    if res ==1:
        print('OK')
    else:
        break

# === end of work ======
print("end of work. thanks. bye")


Overwriting ftp_client.py


#### py043 Питон программа управления шаговым двигателем

лекция 369 по электронике  
pip install pyserial

In [2]:
#coding: utf8

import time 
import serial 

port = 'COM5'
ser = serial.Serial(port, 9600, timeout=1)

#-- функция обмена данными с двигателем
#-- отправляет строку и возвращает принятый ответ
def fn_outin( st_out ): 
    global ser 
    ser.write( st_out.encode() ) 
    st_answ = ser.readline().decode('cp1251')
    #st_answ = ser.readline() 
    return st_answ
    
#-- преобразовать симвов числовую тетраду    
    
def fn_char2tetr( ch ):
    st_hex ='0123456789abcdefABCDEF'  
    pos = st_hex.find( ch ) 
    if pos > 15: 
        pos -= 6 
    return pos

#-- преобразвать 2 hex символа в целое число 
def fn_str2num( st ):
    first_tetr = fn_char2tetr( st[0] )
    second_tetr = fn_char2tetr( st[1] )
    if first_tetr < 0 or second_tetr < 0 : 
        return first_tetr * 16 + second_tetr

#-- преобразовать короткого целого числа в 2 hex символа
def fn_char2str(num):
    st=str(hex(num+512))
    return st[3:]

#-- преобразовать целого числа в 2 hex символа
def fn_int2str(num):
    numh = num/256
    sth = str(hex(numh+512))
    numl=num%256
    stl=str(hex(numl+512))
    st=sth[3:]+stl[3:]
    return st

#-- функция вывода в текстовое поле
def fn_print(st):
    tx_sect.insert(END, st[:-1]+'\n')
    pass

# -- нарисовать форму
from tkinter import *
root=Tk()
# панель с органами управления
pn_left= Frame(root,width=200, bg='darkgreen')
pn_left.pack(side='left',fill='y')

ed_width=10
row_num=1
cl_contr='darkgreen'
## заголовок блока состояния формы
lb_stathdr= Label(pn_left, text='Состояние контроллера:', bg=cl_contr ,fg='white')
lb_stathdr.grid(row=0, column=0, columnspan=2)

## поле команды
ed_cmd=Entry(pn_left,width=ed_width)
ed_cmd.grid(row=row_num, column=0)
lb_cmd = Label(pn_left, text='команда', bg=cl_contr, fg='white')
lb_cmd.grid(row=row_num, column=1)
row_num += 1

## поле состояния
ed_stat=Entry(pn_left,width=ed_width)
ed_stat.grid(row=row_num, column=0)
lb_stat = Label(pn_left, text='состояние', bg=cl_contr, fg='white')
lb_stat.grid(row=row_num, column=1)
row_num += 1


## длительность шага
ed_steplen = Entry(pn_left, width= ed_width)
ed_steplen.grid(row=row_num, column=0)
lb_steplen = Label(pn_left, text= 'время шага(2ms)', 
bg= cl_contr,fg='white')
lb_steplen.grid(row=row_num, column=1)
row_num += 1

## число шагов
ed_stepcount = Entry(pn_left, width= ed_width)
ed_stepcount.grid(row=row_num, column=0)
lb_stepcount = Label(pn_left, text= 'число шагов', 
bg= cl_contr,fg='white')
lb_stepcount.grid(row=row_num, column=1)
row_num += 1


## кнопка для чтения состояния
def fn_readstate(event):
    ed_cmd.delete(0, END)
    ed_stat.delete(0, END)
    ed_steplen.delete(0, END)
    ed_stepcount.delete(0, END)

    st_out="r011608:\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print st_in

    cmd_drv=st_in[0:2]
    fn_print('cmd_drv =' + cmd_drv + '')
    if cmd_drv=='01':
        st_cmd = 'вправо'
    elif cmd_drv=='02':    
        st_cmd = 'влево'
    else:    
        st_cmd = 'стоп'
    ed_cmd.insert(END, st_cmd)    

    stat_drv=st_in[3:5]
    #print('stat_drv=',stat_drv)

    step_countl=st_in[6:8]
    #print 'step_countl=',step_countl
    step_low=fn_str2num(step_countl)

    step_counth=st_in[9:11]
    #print 'step_counth=',step_counth
    step_high=fn_str2num(step_counth)
    step_count=step_high*256+step_low
    ed_stepcount.insert(END,str(step_count))

    step_len=st_in[12:14]
    #print 'step_len=',step_len

## заголовок блока управления
lb_uprav=Label(pn_left,text='Управление контроллером:',
bg=cl_contr,fg='white')
lb_uprav.grid(row=row_num, column=0, columnspan=2)
row_num += 1

## направление длижения
lb_dir=Label(pn_left,text='команда:',
bg=cl_contr,fg='white')
lb_dir.grid(row=row_num,column=0)

## кнопка для команды СТОП
def fn_stop(event):
    st_out="w011600:\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print st_in

    bt_stop = Button(pn_left, text = 'СТОП')
    bt_stop.bind("<Button-1>", fn_stop ) 
    bt_stop.grid(row = row_num, column = 1) 
    row_num += 1

## кнопка для команды ВЛЕВО 
def fn_left(event):
    st_out="w011602:\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print st_in

    bt_left = Button(pn_left, text = 'ВЛЕВО')
    bt_left.bind("<Button-1>", fn_left ) 
    bt_left.grid(row = row_num, column = 1) 
    row_num += 1

## кнопка для команды ВПРАВО 
def fn_right(event):
    st_out="w011601:\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print st_in

    bt_right = Button(pn_left, text = 'СТОП')
    bt_right.bind("<Button-1>", fn_right ) 
    bt_right.grid(row = row_num, column = 1) 
    row_num += 1

## кнопка для команды В НАЧАЛО 
def fn_stop(event):
    st_out="w011600:\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print st_in

    bt_stop = Button(pn_left, text = 'СТОП')
    bt_stop.bind("<Button-1>", fn_stop ) 
    bt_stop.grid(row = row_num, column = 1) 
    row_num += 1

## задать число шагов
    lb_setstepcount=Label(pn_left,text='число шагов', b=cl_contr,fg='white')
    lb_setstepcount.grid(row=row_num, column=0)
    ed_setstepcount=Entry(pn_left,width=ed_width)
    ed_setstepcount.grid(row=row_num, column=1)
    row_num+=1

def fn_setstepcount(event):
    step_count=int(ed_setstepcount.get())
    st_stepcount+fn_int2str(step_count)
    st_out="d0118"+st_stepcount+"\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print(st_in)

    bt_setstepcount = Button(pn_left, text = 'СТОП')
    bt_setstepcount.bind("<Button-1>", fn_setstepcount ) 
    bt_setstepcount.grid(row = row_num, column = 1) 
    row_num += 1

## задать время шага в мс
    lb_setsteptime=Label(pn_left,text='время шага 2мс', b=cl_contr,fg='white')
    lb_setsteptime.grid(row=row_num, column=0)
    ed_setsteptime=Entry(pn_left,width=ed_width)
    ed_setsteptime.grid(row=row_num, column=1)
    row_num+=1

def fn_setsteptime(event):
    step_time=int(ed_setsteptime.get())
    st_steptime+fn_int2str(step_count)
    st_out="w011a"+st_steptime+"\n"
    fn_print(st_out)
    st_in=fn_outin(st_out)
    #print(st_in)

    bt_setsteptime = Button(pn_left, text = ' задать ')
    bt_setsteptime.bind("<Button-1>", fn_setsteptime ) 
    bt_setsteptime.grid(row = row_num, column = 1) 
    row_num += 1

## 
## 
## 
## 

#-- правая панель для текста сектора
pn_right =Frame(root,width=800)
pn_right.pack(side='right',fill='both', expand=1)

##-- поле для вывода текста в правой панели
tx_sect = Text(pn_right, height = 35, width = 84) 
tx_sect.grid(row = 0, column = 0)

sb_hor = Scrollbar(pn_right, command = tx_sect.xview, 
orient = HORIZONTAL)
sb_hor.grid(row = 1, column = 0, sticky = W+E+N+S)

sb_ver = Scrollbar(pn_right, command = tx_sect.yview,
orient = VERTICAL)
sb_ver.grid(row = 0, column = 1, sticky = W+E+N+S)

tx_sect.config(yscrollcommand = sb_ver.set, 
xscrollcommand = sb_hor.set, wrap = NONE)

#-- запустить окно программы
root.mainloop()
    


SerialException: could not open port 'COM5': PermissionError(13, 'Отказано в доступе.', None, 5)

#### py044 Питон многопоточные клиент и сервер UDP

```
подключение команд, выполняемых на сервере
типа get_file,
чтобы это делал сам сервер по команде клиента через ftp
кроме того, предусмотреть запуск приложений на сервере ( zb - excel)

версия 6:
вводимая команда должна быть из 4х символов
начианться с 
	'c' - если это команда для клиента
	's' - если для сервера
завершаться должна ":"
в промежутке две буквы типа "сделать что"
например 
	'sgf:' - get_file
		приказ серверу скачать файл с фтп 
	'сpf:' - put_file
		приказ клиенту загрузить файл на  фтп 



получив команду,сервер просматривает словарь команд cmd_dict
сейчас там есть 3 команды от клиента
'st:':servfn.repl_text,  - ответить на текст 
'se:':servfn.repl_echo , - отправить эхо текста
'sc:':servfn.repl_cmd   - выполнить команду на сервере

это множество команд можно расширить
'sg:':servfn.get_file   - загрузить файл с ftp на сервер
'sp:':servfn.put_file   - выгрузить файл с сервера на ftp
'ss:':servfn.stop_work  - остановить работу сервера
'sr:':servfn.run_prog   - запустить на сервере приложение
'so':servfn.open_file   - открыть файл на сервер

```

In [17]:
from socket import *
# loc_ip = gethostbyname(gethostname())
# print(loc_ip, gethostname())
# gethostbyname_ex(socket.gethostname())
# dir(socket)

In [18]:

gethostbyname_ex(gethostname())

('argonpc',
 [],
 ['192.168.0.112', '192.168.56.1', '192.168.189.1', '192.168.19.1'])

In [23]:
%%writefile py44servfn.py
# === servfn.py ====
# coding: cp1251 
from socket import *
import os
import sys
import ftplib
from time import ctime

# === form server socket ====
HOST = ''
PORT = 3000
BUFSIZE = 1024
SOCKADDR = (HOST,PORT)
uServSock = socket(AF_INET,SOCK_DGRAM)
uServSock.bind(SOCKADDR)

loc_ip = gethostbyname(gethostname())        #-- get self IP
# === handling functions ==== 
def get_file(param, addr):
    file_name = param[3:]
    ftp_host = addr[0]
    ftp = ftplib.FTP(ftp_host)
    ftp.login("user", "12345")
    try:
        ftp.retrbinary("RETR " + file_name, open(file_name, "wb").write)
        ftp.quit()
        answ = 'get_file: ' + file_name 
    except:
        answ = "ftp error:  :(,  file: " + file_namr +" not got "
    print (answ)
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
#    uServSock.sendto( '%s' %(answ), addr)
    return 1

def put_file(param, addr):
    file_name = param[3:]
    file_name = loc_ip.replace('.','_') + '_'+file_name
    ftp_host = addr[0]
    ftp = ftplib.FTP(ftp_host)
    ftp.login("user", "12345")
    try:
        ftp.storbinary("STOR " + file_name, open(file_name, "rb"), 1024)
        ftp.quit()
        answ = 'put_file: ' + file_name 
    except:
        answ = "ftp error:  :(,  file: " + file_namr +" not put "
    print (answ)
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def run_file(param, addr):
    answ = 'server run file: ', param[3:]
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def stop_server(param, addr):
    answ = 'server stopped, by  :) '
    servfn.uServSock.sendto( ('%s' %(answ)).encode(), addr)
    servfn.uServSock.close()
    return -1

def repl_echo(param, addr):
    answ = 'echo '+ param[3:]
    s='%s' %(answ)
    uServSock.sendto( s.encode(), addr)
    return 1

def repl_text(param, addr):
    print (param[3:])
    answ = 'ok'
    s='%s' %(answ)
    uServSock.sendto( s.encode(), addr)
    #uServSock.sendto( '%s' %(answ), addr)
    return 1



Overwriting py44servfn.py


In [21]:
%%writefile py44serv06.py
# coding: cp1251 
# === serv06.py ====
# --- example of work with ftp 

from socket import *
import time
import threading

import py44servfn
# === handling functions dictionary ==== 
cmd_dict = {'st:':py44servfn.repl_text, 'se:':py44servfn.repl_echo , 
            'pf:':py44servfn.put_file,     'gf:':py44servfn.get_file,}

# === first thread for broadcast===
#loc_ip = gethostbyname(gethostname())        #-- get self IP
print (py44servfn.loc_ip)
brd_adr = "192.168.1.255"

sk_ip = socket(AF_INET, SOCK_DGRAM, IPPROTO_UDP) 
sk_ip.setsockopt(IPPROTO_IP, IP_MULTICAST_TTL, 2)

def send_locip(interval, loc_ip):
    while 1:
        sk_ip.sendto( ' '.encode(), ( brd_adr, 3010))
        print (py44servfn.loc_ip + ' to '+ brd_adr)
        time.sleep(interval)

tr_ip = threading.Thread( target = send_locip, args =(5.0, py44servfn.loc_ip))
tr_ip.daemon = True
tr_ip.start()


# === main block ====
print ('serv06  waiting:')
while True:
    data,addr = py44servfn.uServSock.recvfrom(py44servfn.BUFSIZE)
    print('aaa',data)
    data = data.decode('cp1251') 
    print(data)
    cmd = data[0:3]
    if cmd in cmd_dict.keys():
        func = cmd_dict[cmd]
        res = func(data, addr)
        if res == -1:   
            break
    else:
        answ = 'unknown command  :( '
        py44servfn.uServSock.sendto( answ.encode(), addr)
# === end of work ====
print ('server stopped ')


Overwriting py44serv06.py


In [ ]:
192.168.56.1 to 10.255.255.255

In [19]:
%%writefile py44clifn.py
# === clifn.py ====
# coding: cp1251 
from socket import *
import os
import sys
import ftplib

# === form client socket ====
LOCKHOST = '127.0.0.1'        #-- for ftp on the localhost
HOST = '127.0.0.1'
PORT = 3000
BUFSIZE = 1024
SOCKADDR = (HOST,PORT)
uCliSock = socket(AF_INET,SOCK_DGRAM)
uCliSock.settimeout(5)
servip_set = set()

# === handling functions ==== 
def set_addr(param):
    global HOST
    global SOCKADDR
    global PORT
    HOST = param[3:]
    SOCKADDR = (HOST,PORT)
    print ("new_host: ", HOST)
    return 1

def send_command(param):
    #print 'send_text: ', param 
    uCliSock.sendto(param, SOCKADDR)
    try:
        answ, addr = uCliSock.recvfrom(BUFSIZE)
    except: 
        answ = 'not answer  :('
    print (SOCKADDR[0], " ", answ)
    return 1

def multisend_command(param):
    global HOST
    global SOCKADDR
    global PORT

    ip_list = list(servip_set)
    if len(ip_list) > 0:
        for ip in ip_list:
            HOST = ip
            SOCKADDR = (HOST,PORT)
            uCliSock.sendto(param.encode(), SOCKADDR)
            try:
                answ, addr = uCliSock.recvfrom(BUFSIZE)
            except: 
                answ = 'not answer  :('
            print (SOCKADDR[0], " ", answ)
    else:
        print ("no servers alive  :(")
    return 1

def help():
    print ("you can input: addres, text, command of filename")
    print ("for example:")
    print ("csa:10.1.2.13          - set addres for contact")
    print ("sst:privet medved      - send text for selected ip")
    print ("sse:privet             - send text with echo")
    print ("spf:1.txt              - put file from server to ftp")
    print ("sgf:2.txt              - get file from ftp to server")
    print ("sew:2.txt              - end work of selected server")
    print ("")
    print ("mst:privet medved      - send text to all servers")
    print ("mse:privet             - send text with echofor all servers")
    print ("mpf:1.txt              - put file from all servers to ftp")
    print ("mgf:2.txt              - get file from ftp to all servers")
    print ("")
    print ("cpf:3.txt              - put file from client to ftp")
    print ("cgf:4.txt              - get file from ftp to client")
    print ("cli:                   - disp list of servers ip")
    print ("cew:                   - end of client work")

def put_file(param):            #-- FTP server on the client host
    print ('put_file: ', param) 
    file_name = param[3:]
    ftp = ftplib.FTP(LOCKHOST)
    ftp.login("user", "12345")
    try:
        ftp.storbinary("STOR " + file_name, open(file_name, "rb"), 1024)
        ftp.quit()
    except:
        print ("ftp error:  :(")
        return -1
    return 1

def get_file(param):
    print ('get_file: ', param) 
    file_name = param[3:]
    ftp = ftplib.FTP(LOCKHOST)
    ftp.login("user", "12345")
    try:
        ftp.retrbinary("RETR " + file_name, open(file_name, "wb").write)
        ftp.quit()
    except:
        print ("ftp error:  :(")
        return -1
    return 1

def list_ip(param):
    ip_list = list(servip_set)
    print ("active ips:")
    for ip in ip_list:
        print (ip)
    return 1

def end_work(param):
    uCliSock.close() 
    return -1

def list_files(param):
    print ('list_files: ', param )
    return 1


Overwriting py44clifn.py


In [24]:
%%writefile py44cli06.py
# coding: cp1251 
# === cli06.py ====
# --- example sending command to server and more help
from socket import * 
import threading
import time

import py44clifn

# === handling functions dictionary ==== 
client_cmd = {'sa:':py44clifn.set_addr, 'pf:':py44clifn.put_file, 'lf:':py44clifn.list_files, 
            'gf:':py44clifn.get_file, 'li:':py44clifn.list_ip, 'ew:':py44clifn.end_work }

# == thread for get clients IP ====
#servip_set = {}
broad_sock = socket(AF_INET, SOCK_DGRAM)
client_ip = '';   UDP_PORT = 3010
broad_sock.bind((client_ip, UDP_PORT))

def get_servip():
    while True:
        data, addr = broad_sock.recvfrom(1024)
        # print (data, addr[0])
        py44clifn.servip_set.add(addr[0])
        time.sleep(0.001)

tr_ip = threading.Thread( target = get_servip)
tr_ip.daemon = True
tr_ip.start()


# === main block ====
print ("cli06.py")
while True:
    data = input('>')
    if len(data)<4:  data =data +':'
    dest = data[0]                    #-- letter c or s or m 
    #print ('dest = ', dest)
    if  dest == 's':                    #-- command for server
        print ('send command: ', data[1:])
        res = py44clifn.send_command(data[1:])
        if res == -1:  break
    elif  dest == 'm':                #-- send command to all servers
        print ('multisend command: ', data[1:])
        res = py44clifn.multisend_command(data[1:])
        if res == -1:  break
    elif  dest == 'c':                #-- command for client    
        cmd = data[1:4]                #-- command
        #print ('cmd = ', cmd)
        if cmd in client_cmd.keys():
            func = client_cmd[cmd]
            res = func(data[1:])
            if res == -1:  break
        else:
            print ("client command error  :( ")
            py44clifn.help()         
    else:
        print ("command error  :( ")
        py44clifn.help()    

# === end of work ====
print ("end of work.  thanks.  bye")


Overwriting py44cli06.py


#### py045 Питон система ведущий-ведомые в локальной сети

```
Описание работы программного комплекса cli007.py + serv07.py

комплекс предназначен для удаленного управления компьютерами
при проведении тестирования студентов

он состоит из трех групп программ
	cli07.py, clifn.py - устанавливаются на ведущем компьютере
	serv07.py, servfn.py - устанавливаются на каждом  из всех компьютерах, которыми требуется управлять(ведомых)
	triv_ftp.py - праграмма сервера FTP для передачи файлов,
	устанавливается на ведущем компьютере

Дополнительно на все компьютеры должен быть установлен Python27,
а на ведущий также библиотека ftpdlib для сервера FTP

Работа комплекса управляется ведущим компьютером, для чего достаточно
запустить cli07.py. На каждом ведомом компе надо запустить serv07.py.
Также на вндущей машине нужно запустить triv_ftp.py. В отладочных целях все программы работают в консольном режиме. 

После запуска serv07.py каждая ведомвя машина начинает испускать широковещательные UDP пакеты,, по которым сервер узнает о ее включении.

cli07.py, запущеный на ведущем компьютере,, выдает приглашение ">" 
и переходит в режим ожидания команды. Эти команды подразделяются на 3 класса: клиентские, серверные и множественные. Их можно отличить по первому символу команды( соответственно: "c", "s", "m")

каждая команда однотипна по структуре: 
	управляющая часть(3 символа)
	символ-разделитель - ":"
	содержательная часть команды

Управляющая часть команды состоит из символа класса( "c", "s", "m")
и двух символов, обозначающих действие. Возможны следующие команды:

	sst:privet medved      - send text for selected ip
								послать текст выбранному серверу
	sse:privet             - send text with echo
								послать текст серверу с эхо ответом
	spf:1.txt              - put file from server to ftp
								загрузить файл на ФТП с сервера
	sgf:2.txt              - get file from ftp to server"
								прочитать файл с ФТП на сервер
	sof:1.xls              - open file on the server"
								открыть экселевский файл на сервере
	scf:                   - close opened file on the server"
								закрыть экселевский файл на сервере
	sew:                   - end work of selected server"
								завершить работу сервера
	sgd:				   - get directory name of server
								получить имя текущего каталога на сервере
	slf:				   - get list of files on servers directory
								получить список файлов текущего каталога сервера
	sld:				   - get list of subdirectories on server
								получить список подкаталогов текущего каталога сервера
	scd:new_dir			   - change work directory on the server
								изменить текущий каталог на сервере

Замена первой буквы "s" на "m" в серверных командах заствляет выполнить их на всех серверах. Например:
	mof:1.xls              - open file on every server"
								открыть экселевский файл на всех серверах
Клиентские команды:
	cli:                   - disp list of servers ip
								показать список IP живых серверов
	csa:10.1.2.13          - set addres for contact
								установить адрес сервера(выбрать его)
	cpf:3.txt              - put file from client to ftp"
								загрузить файл на ФТП с клиента
	cgf:4.txt              - get file from ftp to client"
								прочитать файл с ФТП на клиент
	cew:                   - end of client work"
								завершить работу клиента


```

In [2]:
%%writefile py45servfn.py
# === servfn.py ====
# coding: cp1251 
from socket import *
import os
import sys
import ftplib
import time 
import random

import fnmatch
import subprocess as sp
# === form server socket ====
HOST = ''
PORT = 3000
BUFSIZE = 1024
SOCKADDR = (HOST,PORT)
uServSock = socket(AF_INET,SOCK_DGRAM)
uServSock.bind(SOCKADDR)

loc_ip = gethostbyname(gethostname())        #-- get self IP

# === find excel location ==== 
path = "c:\\Program Files\\Microsoft Office"
matches = []
for root, dirnames, filenames in os.walk('\\Program Files\\Microsoft Office'):
    for filename in fnmatch.filter(filenames, 'excel.exe'):
        matches.append(os.path.join(root, filename))
excel = '' if len(matches) == 0 else matches[0]
'''
if len(excel)<10:
    print 'can not find excel'
else:
    print 'excel: ', excel 
'''
child = ''
excel_file = ''
# === handling functions ==== 
def get_file(param, addr):
    file_name = param[3:]
    ftp_host = addr[0]
    ftp = ftplib.FTP(ftp_host)
    ftp.login("user", "12345")
    try:
        ftp.retrbinary("RETR " + file_name, open(file_name, "wb").write)
        ftp.quit()
        answ = 'get_file: ' + file_name 
    except:
        answ = "ftp error:  :(,  file: " + file_namr +" not got "
    print (answ)
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def put_file(param, addr):
    file_name = param[3:]
    serv_file = loc_ip.replace('.','_') + '_'+file_name
    print (file_name)
    ftp_host = addr[0]
    ftp = ftplib.FTP(ftp_host)
    ftp.login("user", "12345")
    try:
        ftp.storbinary("STOR " + serv_file, open(file_name, "rb"), 1024)
        ftp.quit()
        answ = 'put_file: ' + file_name 
    except:
        answ = "ftp error:  :(,  file: " + file_name +" not put "
    print (answ)
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def run_file(param, addr):
    fil_name = param[3:]
    print (fil_name)
    try:
        os.startfile( fil_name )
        answ = 'server run file: '+ fil_name
    except:
        answ = 'cannot  open '+ fil_name
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def get_dir(param, addr):
    dir_name = os.getcwd()
    answ = 'server directory: ' + dir_name
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def list_files(param, addr):
    files = [f for f in os.listdir('.') if os.path.isfile(f)]
    file_list = ''
    for f in files:
        file_list += f + '\n'
    answ = 'server files: \n' + file_list
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def list_dirs(param, addr):
    files = [f for f in os.listdir('.') if os.path.isdir(f)]
    file_list = ''
    for f in files:
        file_list += f + '\n'
    answ = 'server subdirectories: \n' + file_list
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def change_dir(param, addr):
    new_dir = param[3:]
    print (new_dir)
    try:
        os.chdir(new_dir)
        dir_name = os.getcwd()
        answ = 'new directory: ' + dir_name
    except:
        dir_name = os.getcwd()
        answ = 'current directory: ' + dir_name
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def repl_echo(param, addr):
    answ = 'echo '+ param[3:]
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def repl_text(param, addr):
    print (param[3:])
    answ = 'ok'
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def open_file(param, addr):
    global child
    global excel_file
    excel_file = param[3:]
    try:
        child = sp.Popen([excel, excel_file])
        if child.poll() == None:
            answ = 'server opened file: '+ excel_file
        else:
            answ = 'server cannot open file: '+ excel_file
    except:
        print ("cannot open file: "+ excel_file)
        answ = 'server cannot open file: '+ excel_file

    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def close_file(param, addr):
    global child
    global excel_file
    try:
        if child.poll() == None:
            child.terminate()
        while child.poll() == None:
            time.sleep(0.010)
        answ = 'closed file: '+ excel_file
    except:
        print ("cannot close file: "+ excel_file)
        answ = 'server cannot close file: '+ excel_file
    uServSock.sendto( ('%s' %(answ)).encode(), addr)
    return 1

def send_ip(param, addr):
    loc_time = random.random()
    time.sleep(loc_time)
    uServSock.sendto( ('IP:').encode(), addr)
    return 1


Overwriting py45servfn.py


In [3]:
%%writefile py45serv07.py
# coding: cp1251 
# === serv07.py ====
# --- example of work with excel files 

from socket import *
import time
import py45servfn


def end_work(data, addr):
        answ = 'server stopped '
        print (answ)        
        py45servfn.uServSock.sendto( answ.encode(), addr)
        return  -1

# === handling functions dictionary ==== 
cmd_dict = {'st:':py45servfn.repl_text,  'se:':py45servfn.repl_echo,
            'ew:':end_work, 'gd:':py45servfn.get_dir,'lf:':py45servfn.list_files, 
            'ld:':py45servfn.list_dirs, 'cd:':py45servfn.change_dir,
            'pf:':py45servfn.put_file,   'gf:':py45servfn.get_file,
            'of:':py45servfn.open_file,  'cf:':py45servfn.close_file,
            'si:':py45servfn.send_ip,'rf:':py45servfn.run_file }

# === main block ====
print ('serv07  waiting:')
while True:
    data,addr = py45servfn.uServSock.recvfrom(py45servfn.BUFSIZE)
    data = data.decode('cp1251') 
    cmd = data[0:3]
    print (data)
    if cmd in cmd_dict.keys():
        func = cmd_dict[cmd]
        res = func(data, addr)
        if res == -1:   
            break
    else:
        answ = 'unknown command  :( '
        py45servfn.uServSock.sendto( ('%s' %(answ)).encode(), addr)
# === end of work ====



Overwriting py45serv07.py


In [8]:
%%writefile py45serv07.bat
python py45serv07.py

Overwriting py45serv07.bat


In [13]:
%%writefile py45clifn.py
# === clifn.py ====
# coding: cp1251 
from socket import *
import os
import sys
import ftplib
import time
import threading

# === form client socket ====
LOCKHOST = '127.0.0.1'        #-- for ftp on the localhost
HOST = '127.0.0.1'
PORT = 3000
BUFSIZE = 1024
SOCKADDR = (HOST,PORT)
uCliSock = socket(AF_INET,SOCK_DGRAM)
uCliSock.settimeout(5)
servip_set = set()

time_beg = 0
time_cur = 10

def get_servip():
    while True: 
        if time_cur - time_beg < 5:
            try:
                data, addr = uCliSock.recvfrom(BUFSIZE)
                if data[:2].decode() == 'IP': 
                    servip_set.add(addr[0])
                    print (addr[0])
            except:
                pass
        time.sleep(0.003)

tr_ip = threading.Thread( target = get_servip)
tr_ip.daemon = True
tr_ip.start()

# === handling functions ==== 
def set_addr(param):
    global HOST
    global SOCKADDR
    global PORT
    HOST = param[3:]
    SOCKADDR = (HOST,PORT)
    print ("new_host: ", HOST)
    return 1

def send_command(param):
    #print 'send_text: ', param 
    uCliSock.sendto(param.encode(), SOCKADDR)
    try:
        answ, addr = uCliSock.recvfrom(BUFSIZE)
    except: 
        answ = 'not answer  :('
    print (SOCKADDR[0], " ", answ.decode())
    return 1

def multisend_command(param):
    global HOST
    global SOCKADDR
    global PORT
    global servip_set
    global time_beg
    global time_cur

    if param[:2] == 'si':  #-- запрос активных серверов
        servip_set.clear()
        brd_adr = "192.168.0.255"
        HOST = brd_adr
        SOCKADDR = (HOST,PORT)
        uCliSock.sendto(param.encode(), SOCKADDR)
        time_beg = time.time()
        time_cur = time.time()
        while time_cur - time_beg < 5:
            time_cur = time.time()
            time.sleep(0.002)
        return 1

    ip_list = list(servip_set)
    if len(ip_list) > 0:
        for ip in ip_list:
            HOST = ip
            SOCKADDR = (HOST,PORT)
            uCliSock.sendto(param.encode(), SOCKADDR)
            try:
                answ, addr = uCliSock.recvfrom(BUFSIZE)
            except: 
                answ = 'not answer  :('
            print (SOCKADDR[0], " ", answ)
    else:
        print ("no servers alive  :(")
        print ("input msi:")

    return 1

def help():
    print ("you can input: addres, text, command of filename")
    print ("for example server commands:")
    print ("sst:privet medved   - send text for selected ip")
    print ("sse:privet          - send text with echo")
    print ("spf:1.txt           - put file from server to ftp")
    print ("sgf:2.txt           - get file from ftp to server")
    print ("sof:1.xls           - open file on the server")
    print ("scf:                - close opened file on the server")
    print ("sgd:                - get directory name of server")
    print ("slf:                - get list of files on servers directory")
    print ("sld:                   - get list of subdirectories on server")
    print ("scd:new_dir         - change work directory on the server")
    print ("sew:                - end work of selected server")
    print ("")
    print ("if You change first character from 's' to 'm', ")
    print ("thee commands wil handle on all servers. for example:")
    print ("mof:1.xls           - open file on every server")
    print ("")
    print ("client commands:")
    print ("msi:                - fill list of servers ip")
    print ("cli:                - disp list of servers ip")
    print ("csa:10.1.2.13       - set addres for contact")
    print ("")
    print ("cgd:                - get directory name of client")
    print ("clf:                - get list of files on clients directory")
    print ("cld:                   - get list of subdirectories on client")
    print ("ccd:new_dir         - change work directory on the client")
    print ("cew:                - end of client work")
    print ("")
    print ("cpf:3.txt           - put file from client to ftp")
    print ("cgf:4.txt           - get file from ftp to client")

def put_file(param):            #-- FTP server on the client host
    print ('put_file: ', param) 
    file_name = param[3:]
    ftp = ftplib.FTP(LOCKHOST)
    ftp.login("user", "12345")
    try:
        ftp.storbinary("STOR " + file_name, open(file_name, "rb"), 1024)
        ftp.quit()
    except:
        print ("ftp error:  :(")
        return -1
    return 1

def get_file(param):
    print ('get_file: ', param )
    file_name = param[3:]
    ftp = ftplib.FTP(LOCKHOST)
    ftp.login("user", "12345")
    try:
        ftp.retrbinary("RETR " + file_name, open(file_name, "wb").write)
        ftp.quit()
    except:
        print ("ftp error:  :(")
        return -1
    return 1

def list_ip(param):
    ip_list = list(servip_set)
    print ("active ips:")
    for ip in ip_list:
        print (ip)
    return 1

def end_work(param):
    uCliSock.close() 
    return -1

def list_files(param):
    files = [f for f in os.listdir('.') if os.path.isfile(f)]
    file_list = ''
    for f in files:
        file_list += f + '\n'
    answ = ('client files: \n' + file_list)
    print (answ) 
    return 1

def get_dir(param):
    dir_name = os.getcwd()
    answ = 'client directory: ' + dir_name
    print (answ) 
    return 1

def list_dirs(param):
    files = [f for f in os.listdir('.') if os.path.isdir(f)]
    file_list = ''
    for f in files:
        file_list += f + '\n'
    answ = 'server subdirectories: \n' + file_list
    print (answ) 
    return 1

def change_dir(param):
    new_dir = param[3:]
    print (new_dir)
    try:
        os.chdir(new_dir)
        dir_name = os.getcwd()
        answ = 'new directory: ' + dir_name
    except:
        dir_name = os.getcwd()
        answ = 'current directory: ' + dir_name
    print (answ) 
    return 1



Overwriting py45clifn.py


In [5]:
%%writefile py45cli07.py
# coding: cp1251 
# === cli07.py ====
# --- example sending command to server and more help

import py45clifn

# === handling functions dictionary ==== 
client_cmd = {'sa:':py45clifn.set_addr, 'pf:':py45clifn.put_file, 'lf:':py45clifn.list_files, 
            'gf:':py45clifn.get_file, 'li:':py45clifn.list_ip, 'ew:':py45clifn.end_work,
             'gd:':py45clifn.get_dir,
            'ld:':py45clifn.list_dirs, 'cd:':py45clifn.change_dir}

# === main block ====
print ("cli07.py")

data = 'msi:'
while True:
    if len(data)<4:  data =data +':'
    dest = data[0]                    #-- letter c or s or m 
    #print 'dest = ', dest
    if  dest == 's':                    #-- command for server
        print ('send command: ', data[1:])
        res = py45clifn.send_command(data[1:])
        if res == -1:  break
    elif  dest == 'm':                #-- send command to all servers
        print ('multisend command: ', data[1:])
        res = py45clifn.multisend_command(data[1:])
        if res == -1:  break
    elif  dest == 'c':                #-- command for client    
        cmd = data[1:4]                #-- command
        #print ('cmd = ', cmd)
        if cmd in client_cmd.keys():
            func = client_cmd[cmd]
            res = func(data[1:])
            if res == -1:  break
        else:
            print ("client command error  :( ")
            py45clifn.help()         
    else:
        print ("command error  :( ")
        py45clifn.help()    
    data = input('>')

# === end of work ====
print ("end of work.  thanks.  bye")


Overwriting py45cli07.py


```>python py45serv07.py
serv07  waiting:
si:
se:prived```
```
>python py45cli07.py
multisend command:  si:
192.168.0.112
>msi:
multisend command:  si:
192.168.0.112
>csa:192.168.0.112
new_host:  192.168.0.112
>sgd:
send command:  gd:
192.168.0.112   b'server directory: D:\\mailCloud\\prjother\\089hosts\\jnotebook
\\pyoop\\prkt'
>scd:\dload
send command:  cd:\dload
192.168.0.112   b'new directory: D:\\dload'
>sgd:
send command:  gd:
192.168.0.112   b'server directory: D:\\dload'
>slf:
send command:  lf:
192.168.0.112   server files:
arduino-1.8.10-windows (1).exe
arduino-1.8.10-windows (2).exe
arduino-1.8.10-windows.exe
backup.rar
CH341SER.zip
f4_osc_480x320-2019-08-23-hex.zip
hafman.rar
pars_playlist.rar
py018 Питон формирование таблицы DBSN.mp4
>sld:
send command:  ld:
192.168.0.112   server subdirectories:
tmp
>mcd:\dload
multisend command:  cd:\dload
192.168.0.112   b'new directory: D:\\dload'
>ccd:\tmp
\tmp
new directory: D:\tmp
>clf:
client files:
1.txt


>sse:prived
send command:  se:prived
192.168.0.112   b'echo prived'

>cpf:1.txt
put_file:  pf:1.txt
Traceback (most recent call last):
  File "py45cli07.py", line 34, in <module>
....
ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечн
ый компьютер отверг запрос на подключение

```

In [ ]:
%%writefile py45

#### py046 Питон парсинг HTML страницы

In [2]:
from urllib.request import urlopen
import codecs
from bs4 import BeautifulSoup

# url = "http://zz.kl.com.ua/"
url = "https://www.agroprombank.com"

fh = urlopen(url)
page2 = fh.read()
fh.close()

soup = BeautifulSoup(page2)
tables = soup.findAll ('table')

div = soup.find('div', {'class':'pane'})
tbl = div. find ('table')

rows = tbl.findAll('tr')
txt = ''
for row_num in range(1,5): 
    row = rows[row_num] 
    cells = row.findAll('td') 
    st = ''
    for cell in cells:
        txt2 = cell.getText() 
        st += txt2 + ':'
    txt+=st+'\n'
print(txt)

 USD/RUP :16.3000:16.3500:
 EUR/RUP :17.8500:18.6500:
 MDL/RUP :0.9310:0.9720:
 UAH/RUP :0.6800:0.7450:



#### py047 Питон работа по Modbus RTU

[modpoll is a command line based Modbus master simulator and test utility](https://www.modbusdriver.com/modpoll.html)  
[Arduino & Modbus habr](https://habr.com/ru/post/249043/)  
[]()  
[]()  
[]()  


#### py048 Консольное приложение для работы по протоколу Modbus RTU

#### py049 Питон управляем Экселем


In [4]:
# pip list
# pip install pywin32 

In [28]:
# %%writefile 1.py
import win32com.client
Excel = win32com.client.Dispatch("Excel.Application")
wb = Excel.Workbooks.Open('D:\\mailCloud\\prjother\\089hosts\\jnotebook\\pyoop\\prkt\\1.xlsm')
sheet = wb.Sheets(1)
val =sheet.Cells(1,1).Value
sheet.Cells(2,4).Value = 24
val2 =sheet.Cells(2,4).Value
vals = [r[0].value for r in sheet.Range("A1:A8")]
Excel.visible = 1
wb.Save()
Excel.Run("SelectColumn")

# Excel.visible = 0

print(val,val2,vals)

# wb.Close()
# Excel.Quit()


1.0 24.0 [1.0, 2.0, 3.0, None, None, None, None, None]


[vba-macros](https://exceltable.com/vba-macros/makros-udaleniya-pustyh-stolbcov)  
```
Option Explicit
Sub SelectColumn()
  Dim i As Long
  Dim diapaz1 As Range
  Dim diapaz2 As Range
Set diapaz1 = Application.Range(ActiveSheet.Range("A1"), _
ActiveSheet.Cells.SpecialCells(xlCellTypeLastCell))
For i = 1 To diapaz1.Columns.Count
If WorksheetFunction.CountA(diapaz1.Columns(i).EntireColumn) = 0 Then
If diapaz2 Is Nothing Then
Set diapaz2 = diapaz1.Columns(i).EntireColumn
Else
Set diapaz2 = Application.Union(diapaz2, diapaz1.Columns(i).EntireColumn)
End If
End If
Next
If diapaz2 Is Nothing Then
MsgBox "Ненайдено ниодного пустого столбца!"
Else
diapaz2.Select
End If
End Sub
```

#### py040 Питон - сырое чтение дисков
#### py041 Питон сетевой обмен по UDP протоколу
#### py042 Передача файлов по сети
#### py043 Питон программа управления шаговым двигателем
#### py044 Питон многопоточные клиент и сервер UDP
#### py045 Питон система ведущий-ведомые в локальной сети
#### py046 Питон парсинг HTML страницы
#### py047 Питон работа по Modbus RTU
#### py048 Консольное приложение для работы по протоколу Modbus RTU
#### py049 Питон управляем Экселем


[abglazov](http://abglazov.rfpgu.ru/)  
[py040 Питон - сырое чтение дисков](https://www.youtube.com/watch?v=wMx3lqGWgUw&list=PLku9se_HAVOp8pSgrIWGIjHLP-EOw_ywq&index=43)  
[]()  
[]()  
[]()  
